In [3]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge
import Outils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DatabaseError: Execution failed on sql 'select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t."source"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id ': ERREUR:  la colonne v1.the_geom n'existe pas
LINE 1: select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as...
                                                     ^


In [2]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes
df=at.identifier_rd_pt(df)

NameError: name 'ct' is not defined

In [507]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032845656',df,[]))

In [508]:
toto.append('TRONROUT0000000032845656')

In [509]:
toto

['TRONROUT0000000032845656']

In [475]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  def affecter_troncon(df):


'TRONROUT0000000129676502'

In [ ]:
tata=at.affecter_troncon(df)

In [ ]:
dico=at.affecter_troncon(df)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000084053019 à 17:23:44 nb ligne traite : 0, nb ligne differente=0


POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [431]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [433]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

AttributeError: 'GeoDataFrame' object has no attribute 'uniques'

In [418]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [419]:
gdf_lignes2.wkt

'LINESTRING (382086.0999999995 6570728.29999999, 382084.8999999997 6570735.399999993, 382084.6999999997 6570736.299999992, 382082.5999999996 6570749.09999999, 382082.2999999995 6570751.29999999, 382082.1999999997 6570751.699999991, 382081.5999999996 6570757.799999991, 382081.3999999997 6570759.499999992, 382081.2999999996 6570759.999999992, 382079.5999999997 6570774.499999993, 382077.0999999996 6570796.799999991, 382076.9999999996 6570798.09999999, 382076.6999999997 6570800.999999993, 382075.4999999996 6570822.299999991, 382075.4999999996 6570825.499999992, 382074.9999999996 6570865.099999991, 382077.7999999997 6570927.199999992, 382084.8999999997 6570987.499999992, 382098.1999999997 6571052.399999992, 382101.5999999996 6571066.599999992, 382105.2999999997 6571082.099999993, 382105.7999999996 6571083.899999992, 382108.9999999997 6571096.699999992, 382117.8999999997 6571134.099999991, 382145.4999999998 6571247.899999994, 382178.3999999997 6571382.899999994, 382182.3999999997 6571399.099

In [420]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [421]:
point.wkt

'POINT (382130.0765237678 6571184.306101616)'

In [422]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [423]:
lignes_possibles

,id,prec_plani,prec_alti,nature,numero,nom_voie_g,nom_voie_d,importance,cl_admin,gestion,...,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt,codevoie_rdpt,assigne_rdpt
TRONROUT0000000129676514,46111,10.0,2.5,Quasi-autoroute,N137,None,None,1,Nationale,DA,...,82,2,POINT(382109 6571096.69999999),3,POINT(382236 6571634.19999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000129676508,46104,10.0,2.5,Quasi-autoroute,N137,None,None,1,Nationale,DA,...,87,3,POINT(382198.9 6571518.89999999),2,POINT(382097.3 6571099.49999999),NaN,NaN,NaN,NaN,None


In [424]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [426]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [427]:
#garder uniquement la valeur la plus proche du centroid
ligne_filtres

,id,prec_plani,prec_alti,nature,numero,nom_voie_g,nom_voie_d,importance,cl_admin,gestion,...,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt,codevoie_rdpt,assigne_rdpt,distance
TRONROUT0000000129676508,46104,10.0,2.5,Quasi-autoroute,N137,None,None,1,Nationale,DA,...,3,POINT(382198.9 6571518.89999999),2,POINT(382097.3 6571099.49999999),NaN,NaN,NaN,NaN,None,19.64515


POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [292]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry))if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']  
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry'] 
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df.drop('id_rdpt',axis=1),gdf_rd_point,op='within') 
l_dans_p_int=gp.sjoin(df.drop('id_rdpt',axis=1),gdf_rd_point_int,op='within')
l_dans_p_final=l_dans_p.loc[~l_dans_p.index.isin(l_dans_p_int.index.tolist())] 

In [293]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df.drop('id_rdpt',axis=1),l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [294]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [295]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [303]:
carac_rd_pt.dtypes

nb_rte_rdpt       int64
nom_rte_rdpt     object
codevoie_rdpt    object
dtype: object

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [322]:
gdf_avec_rd_pt.head(5)

,id,prec_plani,prec_alti,nature,numero,nom_voie_g,nom_voie_d,importance,cl_admin,gestion,...,src_geom,nb_intrsct_tgt,tgt_geom,nb_rte_rdpt_x,nom_rte_rdpt_x,id_rdpt,nb_rte_rdpt_y,nom_rte_rdpt_y,codevoie_rdpt,assigne_rdpt
TRONROUT0000000084053019,41306,2.5,2.5,Rd_pt,D939BIS,None,None,2,Départementale,17,...,POINT(409887.1 6563918.99999999),3,POINT(409883.7 6563917.09999999),3.0,"NC,D115,D939BIS",0.0,3.0,"NC,D115,D939BIS","174340920,NR,174341060",True
TRONROUT0000000032852544,143359,5.0,2.5,Route à 2 chaussées,NC,BD JOFFRE,BD JOFFRE,3,Autre,NC,...,POINT(380413.8 6570173.89999999),2,POINT(380404.1 6570156.29999999),NaN,NaN,NaN,NaN,NaN,NaN,None
TRONROUT0000000032845355,137522,10.0,2.5,Route à 2 chaussées,NC,BD ANDRE SAUTEL,BD ANDRE SAUTEL,3,Autre,NC,...,POINT(381897.9 6571736.99999999),2,POINT(381677.2 6571643.79999999),NaN,NaN,NaN,NaN,NaN,NaN,None
TRONROUT0000000032845118,137331,10.0,2.5,Route à 2 chaussées,NC,None,None,3,Autre,NC,...,POINT(382080.9 6571816.99999999),2,POINT(382040.1 6571799.59999999),NaN,NaN,NaN,NaN,NaN,NaN,None
TRONROUT0000000032863601,151781,1.5,1.0,Route à 1 chaussée,D111,None,None,3,Départementale,17,...,POINT(417180.2 6564380.39999999),2,POINT(417805.2 6564551.39999999),NaN,NaN,NaN,NaN,NaN,NaN,None


In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [337]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'D:\temp\otv\17\test_rd_point\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [284]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

['173006945', '173003140', '173006035']


RTreeError: Error in "Index_CreateWithStream": IllegalArgumentException: RTree::BulkLoader::bulkLoadUsingSTR: Empty data stream given.

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]

In [231]:
liste_ligne_touchees

['TRONROUT0000000032845699',
 'TRONROUT0000000032845656',
 'TRONROUT0000000032845688']

In [440]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() : 
    print(key, value)

nb_intrsct_src ['source', 'src_geom']
nb_intrsct_tgt ['target', 'tgt_geom']


In [478]:
for key, value in [['a','b','c'],['d','e','f']].items() :
    print(key, value)

AttributeError: 'list' object has no attribute 'items'